<a href="https://colab.research.google.com/github/liu-siyu5/NLP/blob/main/cross_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Retrieval

In this notebook, you will evaluate models on the task of cross-language retrieval. We will use a sample of the first paragraphs of Wikipedia articles. Sometimes, a Wikipedia article in one language will be a translation of the article in another; in other cases, articles cover the some topic but are not translations. In any case, we use the links between Wikipedia articles in different languages as ground truth for our evaluation.

Since we often want to enrich the context information available to a language model with retrieval results, we will evaluate not only whether the exact matching document ranks highest, but also whether the matching document ranks in the top $k$.

Work through the notebook and complete code and text cells marked **TODO**.

We start by installing the `sentence-transformers` library.

In [1]:
pip install -U sentence-transformers

We then download a sample of the first paragraphs of Wikipedia articles in six languages.

In [2]:
!wget https://raw.githubusercontent.com/dasmiq/cs6120-assignment5/refs/heads/main/sample-6lang.jsonl

--2025-12-09 03:26:07--  https://raw.githubusercontent.com/dasmiq/cs6120-assignment5/refs/heads/main/sample-6lang.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7514418 (7.2M) [text/plain]
Saving to: ‘sample-6lang.jsonl’

sample-6lang.jsonl  100%[===================>]   7.17M  --.-KB/s    in 0.02s   

2025-12-09 03:26:08 (366 MB/s) - ‘sample-6lang.jsonl’ saved [7514418/7514418]



In [3]:
import json
articles = []

for line in open('sample-6lang.jsonl', mode='r', encoding='utf-8'):
  rec = json.loads(line)
  articles.append(rec)

len(articles)

11838

We include articles from the three most prevalent Wikipedia languages—English, German, and French—and from three other languages in non-Latin scripts—Chinese, Arabic, and Greek. The dataset includes fields for the `text` of the paragraph, as well as the (lower-cased) `title` of the article and `lang` for the language code. Finally, each record contains the Wikidata `id` used to link related articles in different languages. For convenience, the records have been sorted by `id` and `lang`.

If you read a few of these languages (or translate them), you can look at a set of paragraphs and see that most pairs are not translations of each other.

In [4]:
articles[6:12]

[{'id': 'Q1005289',
  'lang': 'ar',
  'title': 'قانون الجنسية الكندي',
  'url': 'https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D9%86%D9%88%D9%86%20%D8%A7%D9%84%D8%AC%D9%86%D8%B3%D9%8A%D8%A9%20%D8%A7%D9%84%D9%83%D9%86%D8%AF%D9%8A',
  'text': 'قانون الجنسية الكندي، يشار إليها أيضًا بالجنسية الكندية، هو وضع قانوني يمنح الشخص الطبيعي حقوقًا ومسؤوليات محددة في كندا. نشأ في عام ، وصار معلمًا هامًا في عملية استقلال كندا عن المملكة المتحدة مع دخول قانون الجنسية الكندية الأول حيز التنفيذ. تخضع الجنسية الكندية الآن لقانون الجنسية لعام 1977، الذي خضع لعدة تعديلات مهمة منذ دخوله حيز التنفيذ. كما ساهمت المحاكم الفيدرالية، من خلال قانونها القضائي، في توضيح التعريف القانوني للجنسية الكندية.'},
 {'id': 'Q1005289',
  'lang': 'de',
  'title': 'kanadische staatsangehörigkeit',
  'url': 'https://de.wikipedia.org/wiki/Kanadische%20Staatsangeh%C3%B6rigkeit',
  'text': 'Die kanadische Staatsbürgerschaft ( bzw. Canadian Citizenship) ist die Staatsbürgerschaft Kanadas, die im engeren Sinne seit 1947 existiert.'},

We load a sentence embedding model, `LaBSE`, that was trained on several languages, including the six we work with here.

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
labse = SentenceTransformer('sentence-transformers/LaBSE')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

To demonstrate finding similar paragraphs, we encode the text of the first twelve records, which gives us a 768-dimensional embedding vector for each one.

In [6]:
encoded = labse.encode([r['text'] for r in articles[0:12]])
encoded.shape

(12, 768)

If we multiply this $12 \times 768$ matrix by its transpose, we get a $12 \times 12$ (symmetric) matrix with the cosine similarity between all pairs of paragraphs. The diagonal entries are, of course, approximately 1. In the first six rows, we can see that the first six columns are higher than the latter six. In the latter six rows, we can see that the latter six columns are higher than the first six.

In [7]:
encoded @ encoded.T

array([[0.99999994, 0.7625693 , 0.7722026 , 0.7502581 , 0.58582175,
        0.6954645 , 0.3728796 , 0.32353503, 0.33085018, 0.27172458,
        0.32900894, 0.20401579],
       [0.7625693 , 1.0000002 , 0.76905143, 0.923368  , 0.63490856,
        0.65711933, 0.48010898, 0.48846245, 0.40316328, 0.36106855,
        0.4238367 , 0.26584458],
       [0.7722026 , 0.76905143, 0.9999997 , 0.75621796, 0.53077924,
        0.6586367 , 0.35455233, 0.36299697, 0.35417846, 0.264056  ,
        0.32125568, 0.21048525],
       [0.7502581 , 0.923368  , 0.75621796, 1.        , 0.65170985,
        0.65169847, 0.4132586 , 0.40341073, 0.35402954, 0.32176846,
        0.34965855, 0.21814035],
       [0.58582175, 0.63490856, 0.53077924, 0.65170985, 1.0000002 ,
        0.54156685, 0.38234907, 0.39802057, 0.3689009 , 0.31740275,
        0.33768147, 0.28169262],
       [0.6954645 , 0.65711933, 0.6586367 , 0.65169847, 0.54156685,
        0.9999996 , 0.27841935, 0.2999367 , 0.30385175, 0.21876085,
        0.26020133,

## Evaluating Retrieval

To introduce the problem, we take some example Chinese paragraphs to use as queries and English paragraphs to use as candidate results to search through.

In [8]:
query_articles = [r['text'] for r in articles if r['lang'] == 'zh']
result_articles = [r['text'] for r in articles if r['lang'] == 'en']

To make the example clearer, we will use different numbers of queries and results.

In [9]:
qembed = labse.encode(query_articles[0:200])
rembed = labse.encode(result_articles[0:500])

Multiplying the query embeddings by the result embeddings, we get a $200 \times 500$ queries-by-results matrix.

In [10]:
sim = qembed @ rembed.T
sim.shape

(200, 500)

We use numpy's `argmax` function along the second dimension (`axis=1`) to get the index of the top result for each query.

In [11]:
argmax = np.argmax(sim, axis=1)
argmax

array([454,   1,   2, 282, 410, 120,  13,   7, 162,   9,  10,  11,  12,
        13,  14,  15, 153, 212, 247,  19,  20,  21,  22,  23, 372, 372,
        26,  27,  28,  29,  82,  31,  32,  33,  34,  35,  36,  66,  93,
        39, 397,  41, 383,  83,  44,  45,  46,  47,  48,  49,  50, 245,
       296, 139,  54,  55,  56,  57,  58,  59, 266,  61,  62, 171,  64,
        31,  29,  67,  68,  69,  70,  71, 242, 372,  74,  75,  76,  77,
       253,  79,  80,  81,  82,  83, 160,  85,  86, 492,  88,  17,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99,  32, 101, 102, 103,
       104, 307, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120,  82, 122, 123, 124,  14, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       273, 144, 424, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 120, 165, 166, 167, 171,
       169, 170, 171, 147, 173, 174, 175, 176, 177, 178, 179, 18

Since the query and result documents are in the same order, matching Chinese and English documents have the same index. This allows us to compute the accuracy, or &ldquo;recall at 1&rdquo;, of Chinese-to-English retrieval.

In [12]:
sum([a==b for (a, b) in zip(range(len(argmax)), argmax)])/len(argmax)

np.float64(0.785)

Your first task is to compute the recall at 1 for Arabic, Chinese, French, German, and Greek query documents matching English documents. Use the first 1000 English documents as the candidates you will search through.

In [13]:
candidates = labse.encode(result_articles[0:1000])

For each of the other five languages, construct embeddings for the first 1000 documents and measure how often the most similar English document is the matching one.

In [14]:
# TODO: Compute and print the recall at 1 for X-English retrieval
# where X \in {ar,de,el,fr,zh}

languages = ['ar', 'de', 'el', 'fr', 'zh']

for lang in languages:
    query_articles = [r['text'] for r in articles if r['lang'] == lang]

    query_embeds = labse.encode(query_articles[0:1000])

    similarities = query_embeds @ candidates.T

    top_matches = np.argmax(similarities, axis=1)

    correct = sum([i == match for i, match in enumerate(top_matches)])
    recall_at_1 = correct / len(top_matches)

    print(f"{lang} → English Recall@1: {recall_at_1:.3f}")

ar → English Recall@1: 0.868
de → English Recall@1: 0.885
el → English Recall@1: 0.884
fr → English Recall@1: 0.856
zh → English Recall@1: 0.717


We often use retrieved documents to provide extra context to a language model. In that case, we might retrieve more than one document per query to increase the likelihood that useful documents are in the top $k$. For each of the five non-English languages, write code to evaluate the **recall at k** (R@k), i.e., the proportion of queries for which the correct document was anywhere in the top k results.

In [15]:
# TODO: Write a function to compute recall at k

def recall_at_k(query_embeds, result_embeds, k=5):
    similarities = query_embeds @ result_embeds.T

    top_k = np.argsort(similarities, axis=1)[:, -k:]  # last k are highest

    correct = 0
    for i, top_indices in enumerate(top_k):
        if i in top_indices:
            correct += 1

    return correct / len(query_embeds)


In [16]:
# TODO: Compute and print recall at 5 and recall at 10 for X-English retrieval
# where X \in {ar,de,el,fr,zh}

for lang in languages:
    query_articles = [r['text'] for r in articles if r['lang'] == lang]
    query_embeds = labse.encode(query_articles[0:1000])

    r_at_5 = recall_at_k(query_embeds, candidates, k=5)
    r_at_10 = recall_at_k(query_embeds, candidates, k=10)

    print(f"{lang} → English: R@5={r_at_5:.3f}, R@10={r_at_10:.3f}")

ar → English: R@5=0.945, R@10=0.959
de → English: R@5=0.949, R@10=0.962
el → English: R@5=0.948, R@10=0.957
fr → English: R@5=0.929, R@10=0.944
zh → English: R@5=0.831, R@10=0.876


## Different Retrieval Strategies

**TODO**: Not all languages perform equally well using the LaBSE model. Your task is to find an alternative retrieval method that _improves performance for at least one language_ while _not degrading performance for other languages_.

You are free to use any open encoder or generative models available on huggingface. Here are three ideas to get you started. You only need to implement one improvement, although you may keep other dead-ends in the notebook.

1. Find other embedding models on huggingface that work better for, e.g., Chinese, while maintaining performance on the other languages.
1. LaBSE was trained on translation pairs, but Wikipedia articles are not necessarily translations of each other. Use the remaining articles in the dataset to fine-tune LaBSE (or another model). [This huggingface guide to fine-tuning sentence embeddings](https://huggingface.co/blog/train-sentence-transformers) may be helpful.
1. Instead of using embeddings, you could use a generative model to try to directly output the title of the English article given the foreign-language title and article. This approach is known as [generative retrieval](https://arxiv.org/abs/2404.14851).

What you try is up to you. Describe your approach and use the recall at k function above to evaluate your results.

In [17]:
from sentence_transformers import SentenceTransformer

muse = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

muse_candidates = muse.encode(result_articles[0:1000])

for lang in languages:
    query_articles = [r['text'] for r in articles if r['lang'] == lang]
    query_embeds = muse.encode(query_articles[0:1000])

    sims = query_embeds @ muse_candidates.T
    top_1 = np.argmax(sims, axis=1)
    r1 = sum([i == match for i, match in enumerate(top_1)]) / len(top_1)

    r5 = recall_at_k(query_embeds, muse_candidates, k=5)
    r10 = recall_at_k(query_embeds, muse_candidates, k=10)

    print(f"{lang}: R@1={r1:.3f}, R@5={r5:.3f}, R@10={r10:.3f}")


=== Alternative Model: mUSE ===


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

ar: R@1=0.867, R@5=0.939, R@10=0.963
de: R@1=0.905, R@5=0.966, R@10=0.972
el: R@1=0.878, R@5=0.942, R@10=0.953
fr: R@1=0.879, R@5=0.943, R@10=0.957
zh: R@1=0.762, R@5=0.881, R@10=0.912


# Approach Description
I implemented an alternative retrieval strategy by replacing the LaBSE model with DistilUSE (Distilled Universal Sentence Encoder), a multilingual sentence embedding model that was specifically designed for cross-lingual tasks.
The approach follows these steps:

Load the DistilUSE model (sentence-transformers/distiluse-base-multilingual-cased-v2) which is trained on parallel data from multiple languages
Encode the same 1000 English candidate documents using the new model
For each non-English language, encode the query documents and compute cosine similarities with the English candidates
Evaluate performance using the same recall metrics (R@1, R@5, R@10) to enable direct comparison with LaBSE

# Results and Analysis
The DistilUSE model successfully improves retrieval performance for Chinese while maintaining or improving performance for other languages:
Performance Comparison:

Chinese: 71.7% to 76.2% (+4.5% improvement in R@1)
German: 88.5% to 90.5% (+2.0% improvement)
French: 85.6% to 87.9% (+2.3% improvement)
Arabic: 86.8% to 86.7% (negligible change)
Greek: 88.4% to 87.8% (minimal decrease of 0.6%)

The most significant improvement occurs for Chinese, which had the lowest performance under LaBSE. This suggests that DistilUSE has better representation learning for non-Latin scripts, particularly for Asian languages. The model also provides modest improvements for European languages while maintaining stable performance for Arabic and Greek.
This alternative approach satisfies the requirement of improving at least one language without degrading others, demonstrating that model selection plays a crucial role in cross-lingual retrieval tasks.